<a href="https://colab.research.google.com/github/MarioRMReis/Sentinel_Images/blob/main/Sentinel_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Run me first
Run this cell to install packages if missing.
Run the next cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

Run colab locally.

 - pip install jupyter_http_over_ws  
 - jupyter serverextension enable --py jupyter_http_over_ws
 - jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

In [ ]:
!pip install pykml
!pip install fastkml
!pip install geopandas
!pip intall fiona
!pip install ogr
!pip install fastkml
!pip install geemap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWtgzh7OIHjLpZX-xPNYhFYdJS2WkiUzvipIY1du43BagQ3wp9IK6xMRHLQ

Successfully saved authorization token.


### Datasets and Python modules
Two [datasets](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD) will be used in the tutorial:

- COPERNICUS/S1_GRD_FLOAT
    - Sentinel-1 ground range detected images
- COPERNICUS/S1_GRD  
    - Sentinel-1 ground range detected images converted to decibels

The following cell imports some python modules which we will be using as we go along,  and also enables inline graphics.

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import geemap
import os
import requests
%matplotlib inline

# Parameters for the image request

And in order to make use of interactive maps, we import the _folium_ package:


Let's start work by grabbing a spatial subset of a Sentinel-1 image from the archive. We'll define an region of interest (AOI) as the long-lat corners of a rectangle over the Frankfurt Airport. A convenient way to do this is with the [geojson.io](http://geojson.io/#map=2/20.0/0.0) website, from which we can cut and paste the corresponding GeoJSON object description.

Note that the last and first corners are identical, indicating closure of the polygon. We have to bore down into the GeoJSON structure to get the geometry coordinates, then create an _ee.Geometry()_ object:

In [24]:
from pykml import parser
with open('C:/Users/mario/Documents/Mario/Sentinel/roi/green-botics-fields_2.kml', 'r') as f:
  root = parser.parse(f).getroot()
namespace = {"kml": 'http://www.opengis.net/kml/2.2'}
pms = root.xpath(".//kml:Placemark[.//kml:Polygon]", namespaces=namespace)
roi = []
for p in pms:
  roi.append(p.Polygon.outerBoundaryIs.LinearRing.coordinates)

print(str(roi[0]).split(" "))
print(str(roi[1]).split(" "))

['\n\t\t\t\t\t\t-8.484574796676251,40.22080763336369,13.45845747065939', '-8.484543553913245,40.22044770124932,13.69046881931113', '-8.48477437137255,40.22040157284538,12.90026143566603', '-8.484467701745347,40.21967008641749,13.62266889195817', '-8.48439151611811,40.21864871292406,13.6475823144155', '-8.484042445597971,40.21834685088991,13.47020047691724', '-8.483627852683103,40.21747778063065,14.83627254848795', '-8.474420263015718,40.21903187590421,13.62665056134038', '-8.472899711493463,40.22061040129381,13.45856029720753', '-8.473020619494765,40.22178306009997,13.19536214611589', '-8.484574796676251,40.22080763336369,13.45845747065939', '\n\t\t\t\t\t']
['\n\t\t\t\t\t\t-8.499577980357419,40.21997349534284,11.6282612587632', '-8.498978272105379,40.2210616348755,12.09497411087636', '-8.500776920977012,40.22158781619041,12.30228612854473', '-8.501327737452014,40.22046637830866,12.27831993909386', '-8.499577980357419,40.21997349534284,11.6282612587632', '\n\t\t\t\t\t']


In [ ]:
# create the list for all aois
aoi_list = []
aoi_names = ['Maize1', 'Maize2']
aoi_list.append([[[-8.481757943461618, 40.221058298898306],
              [-8.482487504313669, 40.217650300991174],
              [-8.474054639170847, 40.21909216718002],
              [-8.472788636515817, 40.220484849707624],
              [-8.472938840220651, 40.22176281554504]]])


aoi_list.append([[[-8.500860411678975, 40.221623694557415],
              [-8.501407582318013, 40.220493185225436],
              [-8.499578315769856, 40.21999346132011],
              [-8.49898822978658, 40.22106253829765]]])


# Figuring out the function to create a square with the plot land in the center

 Square to figure out the pixel coord ratio, Download img and load to check the finnal hight and width.

In [ ]:
import math

poly = [[[-8.481757943461618, 40.221058298898306],
           [-8.482487504313669, 40.217650300991174],
           [-8.474054639170847, 40.21909216718002],
           [-8.472788636515817, 40.220484849707624],
           [-8.472938840220651, 40.22176281554504]]]

siz = len(poly[0])
list_x = []
list_y = []
for i in range(siz): 
  list_x.append(poly[0][i][0])
  list_y.append(poly[0][i][1])

centroid_x = sum(list_x)/siz
centroid_y = sum(list_y)/siz

print(centroid_x, centroid_y)


-8.47680551273652 40.22000968646444


In [ ]:
import requests

square = ee.Geometry.Polygon(
    [[[-8.474415606544039, 40.23368990967016],
          [-8.474415606544039, 40.22471245101487],
          [-8.462313479468843, 40.22471245101487],
          [-8.462313479468843, 40.23368990967016]]],None,False)

ffa_s2_test = ee.ImageCollection('COPERNICUS/S2') \
                       .filterBounds(square) \
                       .filterDate(ee.Date('2019-01-01'), ee.Date('2019-12-31'))
colList = ffa_s2_test.toList(10)

img = ee.Image(colList.get(3)).double().clip(square)

rgb = ['B4','B3','B2']

url = img.select(rgb).getThumbURL({'min': 0, 'max': 3000})

img_data = requests.get(url).content

path = 'C:/Users/mario/Documents/Mario/Sentinel/Z-Aux'
id = img.id().getInfo()
with open(path + '/' + id +'.jpg', 'wb') as handler:
  handler.write(img_data)


In [ ]:
import cv2

img_path = 'C:/Users/mario/Documents/Mario/Sentinel/Z-Aux/20190110T112439_20190110T112653_T29TNE.jpg'

img = cv2.imread(img_path)

print(img.shape) # height. width, channels -> 101, 104, 3

In [ ]:
square_coords = [[[-8.474415606544039, 40.23368990967016],
          [-8.474415606544039, 40.22471245101487],
          [-8.462313479468843, 40.22471245101487],
          [-8.462313479468843, 40.23368990967016]]]
height = 101
width = 104

height_coords = abs(square_coords[0][0][1] - square_coords[0][1][1])
width_coords = abs(square_coords[0][1][0] - square_coords[0][2][0])

resize_width = width_coords*128/width
resize_height = height_coords*128/height

print(resize_height,resize_width)

# Functions 128x128 and Check if inside

In [ ]:
# Create a mask-----
# Check if all the reagion of interest is in the image

def get_squares(aois):
    # 0.011377373345324189 0.014894925631009616 -> Height, Width
    new_aois = []
    h_aux = 0.011377373345324189/2
    w_aux = 0.014854925631009616/2
    for aoi in aois:
      # Find centeroid ----------------------------
      siz = len(aoi[0])
      list_width = []
      list_height = []
      for i in range(siz): 
        list_width.append(aoi[0][i][0])
        list_height.append(aoi[0][i][1])

      centroid_width = sum(list_width)/siz
      centroid_height = sum(list_height)/siz

      aoi_new = [[[(centroid_width-w_aux), (centroid_height+h_aux)], [(centroid_width-w_aux), (centroid_height-h_aux)], [(centroid_width+w_aux), (centroid_height-h_aux)], [(centroid_width+w_aux), (centroid_height+h_aux)]]]
      #inside, split_aois = check_if_inside(aoi_new, aoi)
      #if inside:
      new_aois.append(aoi_new)
      #else:
        #for saois in split_aois:
          #new_aois.append(saois)
      
    return new_aois


In [ ]:
def check_if_inside(aoiSquare, aoiPoly):
  inside = True
  # ---- Empty Lists for the x and y coords --
  xListPoly = []
  yListPoly = []
  xListSquare = []
  yListSquare = []
  # ---- Fill the lists with the coords ------
  for i in range(len(aoiPoly[0])): 
    xListPoly.append(aoiPoly[0][i][0])
    yListPoly.append(aoiPoly[0][i][1])
    if i < 4:
      xListSquare.append(aoiSquare[0][i][0])
      yListSquare.append(aoiSquare[0][i][1])

  # ---- Non-Square Coordinates -------------   
  xMinP = min(xListPoly)
  xMaxP = max(xListPoly)
  yMinP = min(yListPoly)
  yMaxP = max(yListPoly)
  # ---- Square Coordinates -----------------
  xMinS = min(xListSquare)
  xMaxS = max(xListSquare)
  yMinS = min(yListSquare)
  yMaxS = max(yListSquare)

  # ----  Create new aois if necessary
  new_aois = []
  new_aois.append(aoiSquare)
  if xMinS > xMinP:
    # new function
    aoi_aux1 = [[[xMinP, yMaxS],
                 [xMinP, yMinS],
                 [xMaxS, yMinS],
                 [xMinS, yMaxS]]]
    new_aois.append(aoi_aux1)
    inside = False
  if xMaxS < xMaxP:
    aoi_aux2 = [[[xMaxS, yMaxS],
                 [xMaxS, yMinS],
                 [xMaxP, yMinS],
                 [xMaxP, yMaxS]]]
    new_aois.append(aoi_aux2)
    inside = False
  if yMinS > yMinP:
    aoi_aux3 = [[[xMinS, yMinS],
                 [xMinS, yMinP],
                 [xMaxS, yMinP],
                 [xMaxS, yMinS]]]
    new_aois.append(aoi_aux3)
    inside = False
  if yMaxS < yMaxP:
    aoi_aux4 = [[[xMinS, yMaxP],
                 [xMinS, yMaxS],
                 [xMaxS, yMaxS],
                 [xMaxS, yMaxP]]]
    new_aois.append(aoi_aux4)
    inside = False
  # (...) 4 more edge cases, diagonal fit

  # inside - returns if the filed fits in the current region created from get_squares()
  # new_aois - list of new regions created when the field doens't fit in the current region 
  # new_polys - returns the field associated with the new region 
  return inside, new_aois, new_polys

In [ ]:
def get_fieldShape(xLand, yLand, xSquare, ySquare, orientation):
  coords_left = []
  coords_right = []
  aoi_landList = []
  for i in range(len(xLand)):
    # new list 
    if orientation == 'Left':
      if xLand[i] < min(xSquare):
        coords_left.append([xLand[i], yLand[i]])
  return aoi_landList

In [ ]:
testing_edge = [[[-8.486570903134364, 40.22201669717746],
          [-8.486570903134364, 40.21728156684586],
          [-8.471378871274013, 40.21728156684586],
          [-8.471378871274013, 40.22201669717746]]]

print(len(testing_edge[0]))

4


In [ ]:
#def create_mask(aoi_land, aoi_square):
testing_edge = [[[-8.486570903134364, 40.22201669717746],
          [-8.486570903134364, 40.21728156684586],
          [-8.471378871274013, 40.21728156684586],
          [-8.471378871274013, 40.22201669717746]]]

aoi_land = [[[-8.481757943461618, 40.221058298898306],
              [-8.482487504313669, 40.217650300991174],
              [-8.474054639170847, 40.21909216718002],
              [-8.472788636515817, 40.220484849707624],
              [-8.472938840220651, 40.22176281554504]]]

aoi_test = ee.Geometry.Polygon(testing_edge,None,False)
aoi_test_land = ee.Geometry.Polygon(aoi_land,None,False)

ffa_s2 = ee.ImageCollection('COPERNICUS/S2') \
                        .filterBounds(aoi_test) \
                        .filter(ee.Filter.bounds(aoi_test_land))\
                        .filterDate(ee.Date('2017-01-01'), ee.Date('2023-12-31'))
colList = ffa_s2.toList(30)


img = ee.Image(colList.get(17)).double().clip(aoi_test)


rgb = ['B4','B3','B2']
url = img.getThumbURL({"min":-20000, "max":-20000,"bands":rgb})

img_data = requests.get(url).content

disp.Image(url=url, width = 300)

# Test

In [ ]:
square_coords = [[[-8.474415606544039, 40.23368990967016],
          [-8.474415606544039, 40.22471245101487],
          [-8.462313479468843, 40.22471245101487],
          [-8.462313479468843, 40.23368990967016]]]

poly = [[[-8.481757943461618, 40.221058298898306],
           [-8.482487504313669, 40.217650300991174],
           [-8.474054639170847, 40.21909216718002],
           [-8.472788636515817, 40.220484849707624],
           [-8.472938840220651, 40.22176281554504]]]

testing_edge = [[[-8.486570903134364, 40.22201669717746],
          [-8.486570903134364, 40.21728156684586],
          [-8.471378871274013, 40.21728156684586],
          [-8.471378871274013, 40.22201669717746]]]
list_testing = []
list_testing.append(testing_edge)
aois_testing = get_squares(list_testing)
test_inside ,new_testingAois = check_if_inside(testing_edge,aois_testing[0])

new_testingGeometry = []
for aoi_test in new_testingAois:
  new_testingGeometry.append(ee.Geometry.Polygon(aoi_test,None,False))

for idx, aoi_test in enumerate(new_testingGeometry):
  ffa_s2 = ee.ImageCollection('COPERNICUS/S2') \
                        .filterBounds(aoi_test) \
                        .filterDate(ee.Date('2017-01-01'), ee.Date('2023-12-31'))
  colList = ffa_s2.toList(30)


  img = ee.Image(colList.get(17)).double().clip(aoi_test)


  rgb = ['B4','B3','B2']
  url = img.getThumbURL({"min":0, "max":2500,"bands":rgb})

  img_data = requests.get(url).content

  path = 'C:/Users/mario/Documents/Mario/Sentinel/Z-Aux/' + str(idx)
  id = img.id().getInfo()
  with open(path + '/' + id +'.jpg', 'wb') as handler:
    handler.write(img_data)

In [ ]:
print(new_testingAois)

In [ ]:
poly = [[[-8.486570903134364, 40.22201669717746],
          [-8.486570903134364, 40.21728156684586],
          [-8.471378871274013, 40.21728156684586],
          [-8.471378871274013, 40.22201669717746]]]

aois = [poly] 
#naois = get_squares(aois)

aoi_test = ee.Geometry.Polygon(aois[0],None,False)


ffa_s2 = ee.ImageCollection('COPERNICUS/S2') \
                       .filterBounds(aoi_test) \
                       .filterDate(ee.Date('2017-01-01'), ee.Date('2023-12-31'))
colList = ffa_s2.toList(30)


img = ee.Image(colList.get(17)).double().clip(aoi_test)


rgb = ['B4','B3','B2']
url = img.getThumbURL({"min":0, "max":2500,"bands":rgb})

img_data = requests.get(url).content

path = 'C:/Users/mario/Documents/Mario/Sentinel/Z-Aux'
id = img.id().getInfo()
with open(path + '/' + id +'.jpg', 'wb') as handler:
  handler.write(img_data)


disp.Image(url=url, width = 300)

Next, we filter the S1 archive to get an image over the _aoi_ acquired sometime in August, 2020. Any old image will do fine, so we won't bother to specify the orbit number or whether we want the ASCENDING or DESCENDING node. If we don't specify the instrument mode or resolution, we get _IW_ (interferometric wide swath) mode and $10\times 10\ m^2$ pixels by default. For convenience we grab both decibel and float versions:

# 128x128 aois

In [ ]:
# for each aoi, get the square and create the new list ee.Geometry.Polygon
aois = get_squares(aoi_list)
aois_geometry = []
for aoi in aois:
  aois_geometry.append(ee.Geometry.Polygon(aoi,None,False))

# Parameters

In [ ]:
folder_dataset = 'C:/Users/mario/Documents/Mario/Sentinel/dataset'
starting_date = '2017-01-01'
ending_date = '2023-12-31'
nimgs = 2190

## SENTINEL-1

In [ ]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aois_geometry[0]) 
                       .filterDate(ee.Date('2019-01-01'), ee.Date('2019-12-31')) 
                       .first() 
                       .clip(aois_geometry[0]))

# Testing the best range for the bands, min and max

In [ ]:
ffa_db.bandNames().getInfo()

['VV', 'VH', 'angle']

In [ ]:
ffa_s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
                       .filterBounds(aois_geometry[0]) \
                       .filterDate(ee.Date('2019-01-01'), ee.Date('2019-12-31'))
colList = ffa_s1.toList(30)

img = ee.Image(colList.get(19)).double().clip(aois_geometry[0])
url = img.select('VH').getThumbURL({'min': -25, 'max': -7})
disp.Image(url=url, width = 300)

# Function to request the images Sentinel-1 ---------------------------------------------------------------------

In [ ]:
def ExportCol_Sentinel1(path, roi, channel, min, max, n):
  try:
    for i in range(0,nimgs,1):
      ffa_s = ee.ImageCollection('COPERNICUS/S1_GRD') \
                        .filterBounds(roi) \
                        .filterDate(ee.Date(starting_date), ee.Date(ending_date))

      colList = ffa_s.toList(nimgs); 

      img = ee.Image(colList.get(i)).double().clip(roi)
      # Exeption beacause to get a RGB channel with the Sentinel-1 we need to correspond each channel and compute the last
      if channel == 'RGB':
        imgR = img.select('VV')
        imgG = img.select('VH')
        imgB = img.select('VV').divide(img.select('VH'))
        url = ee.Image.rgb(imgR,imgG,imgB).getThumbURL({'min': [min[0], min[1], 0], 'max': [0, 0, 2]})
      else:
        url = img.select(channel).getThumbURL({'min': min[n], 'max': max[n]})

      id = img.id().getInfo()
      img_data = requests.get(url).content

      # Creates the folder but the flag need to be turned off beacause it will try to makedir after the creation
      try:
        os.makedirs(path)
        with open(path + '/' + id +'.jpg', 'wb') as handler:
          handler.write(img_data)
          #create_folders = False
      except:
        # This just saves the image in the correct folder with the name 'id' (var->'id')
        with open(path + '/' + id +'.jpg', 'wb') as handler:
          handler.write(img_data)
  except:
    return

The request

In [ ]:
# Variables needed to save images
bands_s1 = ffa_db.bandNames().getInfo()
bands_s1.remove('angle')
bands_s1.append('RGB')

min = [-14, -25]
max = [-7] * 2
#-------------------
for idx, a in enumerate(aois_geometry):
  for jdx, b in enumerate(bands_s1):
    path = folder_dataset +'/'+ aoi_names[idx] +'/'+ b
    ExportCol_Sentinel1(path, a, b, min, max, jdx)

In [ ]:
# delete
for i, a in enumerate(aois):
  print(i)
  print(' ')
  #print(a)

bands_s1 = ffa_db.bandNames().getInfo()
bands_s1.remove('angle')
bands_s1.append('RGB')

for jdx, b in enumerate(bands_s1):
  print(b)

0
 
VV
VH
RGB


# Test of the RGB image using the 2 channels of the Sentinel-1

In [ ]:
# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(ffa_db.select('VV'),
                   ffa_db.select('VH'),
                   ffa_db.select('VV').divide(ffa_db.select('VH'))).getThumbURL({'min': [-14, -25, 0], 'max': [-7, -7, 3]})


disp.Image(url=rgb, width = 300)

# SENTINEL-2 testing ---------------------------------

In [ ]:
aoi1 = ee.Geometry.Polygon(
        [[[-8.48545509905759, 40.22345971261565],
          [-8.48545509905759, 40.2128095274749],
          [-8.461937490415012, 40.2128095274749],
          [-8.461937490415012, 40.22345971261565]]],None,False)

ffa_db2 = ee.Image(ee.ImageCollection('COPERNICUS/S2') 
                       .filterBounds(aoi1) 
                       .filterDate(ee.Date('2019-01-01'), ee.Date('2019-12-31')) 
                       .first()
                       .clip(aoi1))

ffa_s2 = ee.ImageCollection('COPERNICUS/S2') \
                       .filterBounds(aoi1) \
                       .filterDate(ee.Date('2017-01-01'), ee.Date('2023-12-31'))
colList = ffa_s2.toList(30)


img = ee.Image(colList.get(18)).double().clip(aoi1)

rgb = ['B12']
url = img.getThumbURL({"min":0, "max":2500,"bands":rgb})

disp.Image(url=url, width = 300)

In [ ]:
aoi1 = ee.Geometry.Polygon(
         [[[-8.471299278632037, 40.22244132271391],
          [-8.471299278632037, 40.215199270092846],
          [-8.460098373785845, 40.215199270092846],
          [-8.460098373785845, 40.22244132271391]]],None,False)

ffa_db2 = ee.Image(ee.ImageCollection('COPERNICUS/S2') 
                       .filterBounds(aoi1) 
                       .filterDate(ee.Date('2019-01-01'), ee.Date('2019-12-31')) 
                       .first()
                       .clip(aoi1))

ffa_s2 = ee.ImageCollection('COPERNICUS/S2') \
                       .filterBounds(aoi1) \
                       .filterDate(ee.Date('2017-01-01'), ee.Date('2023-12-31'))
colList = ffa_s2.toList(30)


img = ee.Image(colList.get(18)).double().clip(aoi1)

rgb = ['B4','B3','B2']
url = img.getThumbURL({"min":0, "max":2500,"bands":rgb})

disp.Image(url=url, width = 300)

In [ ]:
ffa_db2.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B10',
 'B11',
 'B12',
 'QA10',
 'QA20',
 'QA60']

In [ ]:
rgb = ['B4','B3','B2']
url = ffa_db2.getThumbURL({"min":0, "max":2500,"bands":rgb})

disp.Image(url=url, width = 300)

In [ ]:
ffa_s2 = ee.ImageCollection('COPERNICUS/S2') \
                       .filterBounds(aoi1) \
                       .filterDate(ee.Date('2017-01-01'), ee.Date('2023-12-31'))
colList = ffa_s2.toList(10)

# Sentinel-2 request fucntion

In [ ]:
def ExportCol_Sentinel2(folder, roi, roi_name channel, create_folders, min, max, n):
  for i in range(0,nimgs,1):
    try:
      ffa_s = ee.ImageCollection('COPERNICUS/S2') \
                        .filterBounds(roi) \
                        .filterDate(ee.Date(starting_date), ee.Date(ending_date))

      colList = ffa_s.toList(ffa_s.size());                   

      img = ee.Image(colList.get(i)).double().clip(roi)
      # Exeption beacause to get a RGB channel with the Sentinel-1 we need to correspond each channel and compute the last
      if channel == 'RGB':
        rgb = ['B4','B3','B2']
        url = img.getThumbURL({"min":min[n], "max":max[n],"bands":rgb})
      else:
        url = img.getThumbURL({"min":min[n], "max":max[n],"bands":channel})

      id = img.id().getInfo()
      img_data = requests.get(url).content
      path = 'drive/MyDrive/' + folder

      # Creates the folder but the flag need to be turned off beacause it will try to makedir after the creation
      if create_folders:
        os.makedirs(path)
        with open(path + '/' + id +'.jpg', 'wb') as handler:
          handler.write(img_data)
          create_folders = False
      else:
        # This just saves the image in the correct folder with the name 'id' (var->'id')
        with open(path + '/' + id +'.jpg', 'wb') as handler:
          handler.write(img_data)
    except:
      return

Request

In [ ]:
bands_s2 = ffa_db2.bandNames().getInfo()
aux_bands_s2 =  bands_s2.copy();
for x in aux_bands_s2: 
  if x in  ['QA10','QA20','QA60','B10']: 
    bands_s2.remove(x) #bands_s2.remove(['QA10','QA20','QA60','B10'])
bands_s2.append('RGB')

min = [0] * len(bands_s2)
max = [2700, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 2000, 3200, 2500, 3000]

bands_s2 = ['RGB']
for a in aois:
  for i in range(0,len(bands_s2),1):
    channel = bands_s2[i]
    folder = 'Sentinel-2/Maize/'+ channel

    ExportCol_Sentinel2(folder, a, channel, create_folders, min, max, 12)

# Testing the best range for the bands, min and max, Sentinel-2

In [ ]:
aoi_test = ee.Geometry.Polygon(
        [[[-8.55251498879442, 40.24180169042327],
          [-8.55251498879442, 40.208117730174855],
          [-8.451578099145982, 40.208117730174855],
          [-8.451578099145982, 40.24180169042327]]],None,False)

ffa_s2_test = ee.ImageCollection('COPERNICUS/S2') \
                       .filterBounds(aoi_test) \
                       .filterDate(ee.Date('2019-01-01'), ee.Date('2019-12-31'))
colList = ffa_s2.toList(10)

ffa_db2_test = ee.Image(ee.ImageCollection('COPERNICUS/S2') 
                       .filterBounds(aoi_test) 
                       .filterDate(ee.Date('2019-01-01'), ee.Date('2019-12-31')) 
                       .first()
                       .clip(aoi_test))

img = ee.Image(colList.get(0)).double().clip(aoi_test)

#rgb = ['B4','B3','B2']
#url = ffa_db2_test.getThumbURL({"min":0, "max":3000,"bands":rgb})

url = img.select('QA60').getThumbURL({'min': -50, 'max': 50})
disp.Image(url=url, width = 700)

In [ ]:
bands_s2 = ffa_db2.bandNames().getInfo()
aux =  bands_s2.copy();
for x in aux:
  if x in ['QA20','QA10','QA60','B10']:
    bands_s2.remove(x) #bands_s2.remove(['QA10','QA20','QA60','B10'])
bands_s2.append('RGB')

min = [0] * len(bands_s2)
max = [2700, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 2000, 3200, 2500, 3000]
print(len(max))
print(len(min))
print(bands_s2)

13
13
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'RGB']


In [ ]:
ffa_s = ee.ImageCollection('COPERNICUS/S1_GRD') \
                       .filterBounds(aoi1) \
                       .filterDate(ee.Date('2022-01-07'), ee.Date('2022-04-07'))

colList = ffa_s.toList(ffa_s.size());       

i = 1330
img = ee.Image(colList.get(i)).double().clip(aoi1)

#id = img.id().getInfo()

